## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-27-10-48-06 +0000,bbc,'Families can save £200 a month at community s...,https://www.bbc.com/news/articles/c8jwrzvv2kmo...
1,2025-12-27-10-36-23 +0000,wapo,Russian attack pummels Kyiv as Zelensky prepar...,https://www.washingtonpost.com/world/2025/12/2...
2,2025-12-27-10-32-03 +0000,cbc,"Thailand, Cambodia sign new ceasefire deal to ...",https://www.cbc.ca/news/world/thailand-cambodi...
3,2025-12-27-10-30-41 +0000,wapo,Philadelphia lacked bus-tracking signs. ‘Bus S...,https://www.washingtonpost.com/nation/interact...
4,2025-12-27-10-01-53 +0000,nyt,A Dancing Dictator and Bankers in Chains: The ...,https://www.nytimes.com/2025/12/27/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2368/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,29
93,year,13
15,new,10
10,meet,10
139,peace,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
55,2025-12-27-03-00-00 +0000,wsj,The U.S. is fast-tracking power projects for d...,https://www.wsj.com/politics/policy/ai-state-f...,76
99,2025-12-26-21-22-50 +0000,nypost,Trump tells Post there’s a ‘good shot’ at reac...,https://nypost.com/2025/12/26/us-news/trump-te...,76
34,2025-12-27-07-34-21 +0000,nyt,Zelensky Will Meet With Trump Over the Weekend...,https://www.nytimes.com/2025/12/26/world/europ...,74
160,2025-12-26-15-47-53 +0000,cbc,"Zelenskyy, Trump to meet in Florida on Sunday ...",https://www.cbc.ca/news/world/zelenskyy-trump-...,73
27,2025-12-27-08-44-24 +0000,wapo,"Zelensky says he will meet Trump soon, citing ...",https://www.washingtonpost.com/world/2025/12/2...,71


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
55,76,2025-12-27-03-00-00 +0000,wsj,The U.S. is fast-tracking power projects for d...,https://www.wsj.com/politics/policy/ai-state-f...
99,47,2025-12-26-21-22-50 +0000,nypost,Trump tells Post there’s a ‘good shot’ at reac...,https://nypost.com/2025/12/26/us-news/trump-te...
166,45,2025-12-26-15-24-08 +0000,nypost,Dozens of Islamic State suspects plotting Chri...,https://nypost.com/2025/12/26/world-news/dozen...
2,33,2025-12-27-10-32-03 +0000,cbc,"Thailand, Cambodia sign new ceasefire deal to ...",https://www.cbc.ca/news/world/thailand-cambodi...
101,31,2025-12-26-21-12-49 +0000,startribune,"A late-night, tipsy street interview about ICE...",https://www.startribune.com/a-late-night-tipsy...
73,30,2025-12-26-23-49-29 +0000,bbc,Two killed in suspected Palestinian ramming an...,https://www.bbc.com/news/articles/c9d9znp22pyo...
94,29,2025-12-26-21-48-11 +0000,nypost,NYC commuters face OMNY problems just days bef...,https://nypost.com/2025/12/26/us-news/commuter...
121,27,2025-12-26-19-50-20 +0000,nyt,Trump Invited White South Africans to America....,https://www.nytimes.com/2025/12/26/us/trump-af...
175,26,2025-12-26-14-15-36 +0000,bbc,Ex-Malaysia PM Najib Razak given 15-year jail ...,https://www.bbc.com/news/articles/crk7y27vp1eo...
50,24,2025-12-27-03-30-11 +0000,nypost,Trump-Kennedy Center filing $1M lawsuit agains...,https://nypost.com/2025/12/26/us-news/trump-ke...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
